In [ ]:
# Setup scenario
from copy import deepcopy
import json
from pathlib import Path
from time import time

import geopandas as gpd
import fiona
import numpy as np
import pandas as pd
import pycountry
import shapely
from shapely.geometry import Point


from climada.util.api_client import Client
from climada.entity import Entity, Exposures, LitPop
from climada.entity.impact_funcs import ImpactFunc, ImpactFuncSet, ImpfTropCyclone
from climada.hazard import Hazard
from climada.engine import Impact, ImpactCalc
from climada.entity.impact_funcs.storm_europe import ImpfStormEurope
from climada.entity.impact_funcs.trop_cyclone import ImpfSetTropCyclone

import handlers
from handlers import calculate_impact, calculate_impact_function_set, get_exposure_new, get_hazard_new, get_iso3_country_code, get_ref_year
from logger_config import LoggerConfig

logger = LoggerConfig(logger_types=["file"])


notebook_dir = %pwd

# Go up one directory
BASE_DIR = Path(notebook_dir).parent

# Define the paths for exposures and hazards
DATA_DIR = BASE_DIR / "data"
EXPOSURES_DIR = BASE_DIR / "data" / "exposures"
HAZARDS_DIR = BASE_DIR / "data" / "hazards"
REQUIREMENTS_DIR = BASE_DIR / "requirements"
GADM41_EGY_PATH = REQUIREMENTS_DIR / "gadm41_EGY.gpkg"
GADM41_THA_PATH = REQUIREMENTS_DIR / "gadm41_THA.gpkg"
TEMP_DIR = DATA_DIR / "temp" 
ENTITIES_DIR = BASE_DIR / "data" / "entities"

# Available Exposure data types in CLIMADA API for Egypt/Thailand: ['litpop']
# Available Hazard data types in CLIMADA API for Egypt/Thailand: ['river_flood', 'wildfire', 'earthquake', 'flood', 'tropical_cyclone']
# Available climate scenarios for hazard type river_flood/tropical_c in country Egypt/Thailand: ['rcp26', 'historical', 'rcp60', 'rcp85']
# Available time horizons for hazard type river_flood in country Egypt: ['2030_2050', '1980_2000', '2070_2090', '2010_2030', '2050_2070']

country_name = "Thailand"
exposure_type = "litpop"        # Available exposure types for Egypt/Thailand: ['litpop']
hazard_type = "earthquake"     # Available hazard types for Egypt/Thailand: ['river_flood', 'wildfire', 'earthquake', 'flood', 'tropical_cyclone']
climate_scenario = "historical" # Available scenarios for Egypt/Thailand: ['rcp26', 'historical', 'rcp60', 'rcp85']
time_horizon = "2030_2050"      # Available time horizons for Egypt/Thailand: ['2030_2050', '1980_2000', '2070_2090', '2010_2030', '2050_2070'] / 
annual_growth = 1.01

client = Client()

In [ ]:
# List DataTypeInfos
data_type_infos = client.list_data_type_infos()
exposure_data_types = [
    data_type_info.data_type for data_type_info in data_type_infos if data_type_info.data_type_group == 'exposures']
hazard_data_types = [
    data_type_info.data_type for data_type_info in data_type_infos if data_type_info.data_type_group == 'hazard']

print(
    f'Available Exposure data types in CLIMADA API for all countries:\n{exposure_data_types}')
print(
    f'Available Hazard data types in CLIMADA API for all countries:\n{hazard_data_types}')

In [ ]:
# Available country Exposures and Hazard data types
dataset_infos = client.list_dataset_infos(
    properties={
        'country_name': country_name,
    }
)

exposure_data_types = list(set(
    [dataset_info.data_type.data_type for dataset_info in dataset_infos if dataset_info.data_type.data_type_group == 'exposures']))
hazard_data_types = list(set(
    [dataset_info.data_type.data_type for dataset_info in dataset_infos if dataset_info.data_type.data_type_group == 'hazard']))

print(
    f'Available Exposure data types in CLIMADA API for {country_name}: {exposure_data_types}')
print(
    f'Available Hazard data types in CLIMADA API for {country_name}: {hazard_data_types}')

In [ ]:
if hazard_type == 'river_flood' or 'wildfire':
    available_scenarios = list(set([dataset_info.properties['climate_scenario']
                               for dataset_info in dataset_infos if dataset_info.data_type.data_type == hazard_type]))
    available_time_horizons = list(set([dataset_info.properties['year_range']
                                   for dataset_info in dataset_infos if dataset_info.data_type.data_type == hazard_type]))
if hazard_type == 'tropical_cyclone':
    available_scenarios = list(set([dataset_info.properties['climate_scenario']
                               for dataset_info in dataset_infos if dataset_info.data_type.data_type == hazard_type]))
    available_time_horizons = list(set([dataset_info.properties.get(
        'ref_year') for dataset_info in dataset_infos if dataset_info.data_type.data_type == hazard_type]))
if hazard_type == 'earthquake':
    available_scenarios = []
    available_time_horizons = []
if hazard_type == 'flood':
    available_scenarios = []
    available_time_horizons = list(set([dataset_info.properties['year_range']
                                   for dataset_info in dataset_infos if dataset_info.data_type.data_type == hazard_type]))

print(
    f'Available climate scenarios for hazard type {hazard_type} in country {country_name}: {available_scenarios}')
print(
    f'Available time horizons for hazard type {hazard_type} in country {country_name}: {available_time_horizons}')

In [ ]:
# Available country Hazard DatasetInfos
hazard_dataset_infos = client.list_dataset_infos(
    properties={
        "data_type": hazard_type,
        "country_name": country_name,
        # "climate_scenario": climate_scenario,
    }
)
hazard_dataset_infos

In [ ]:
# Get Exposure and Exposure gdf
# Available Exposures: ['litpop']
exposure = get_exposure_new(country=country_name)
exposure_gdf = exposure.gdf
# exposure_gdf

In [ ]:
# Get Hazard
# Available Hazards: ['river_flood', 'tropical_cyclone', 'wildfire', 'flood', 'earthquake']
hazard = get_hazard_new(
    hazard_type=hazard_type,
    scenario=climate_scenario,
    time_horizon=time_horizon,
    country=country_name,
)

In [ ]:
# Get Impact
impact_function_set = calculate_impact_function_set(
    hazard=hazard, impact_function_name="Flood Africa JRC Residential"
)
impact = calculate_impact(exposure, hazard, impact_function_set)

In [ ]:
# ENTITY_FILE_PATH = ENTITIES_DIR / "entity_TODAY_dummy.xlsx"
ENTITY_FILE_PATH = ENTITIES_DIR / "entity_template.xlsx"
entity = Entity.from_excel(ENTITY_FILE_PATH)
entity.check()
entity.impact_funcs.plot()

In [ ]:
ENTITY_FILE_PATH = ENTITIES_DIR / "entity_TODAY_dummy.xlsx"
entity = Entity.from_excel(ENTITY_FILE_PATH)
entity.check()
entity.impact_funcs.plot()

In [ ]:
from climada.util.constants import ENT_TEMPLATE_XLS
ent = Entity.from_excel(ENT_TEMPLATE_XLS)
imp = ImpactCalc(
    ent.exposures, 
    ent.impact_funcs,
    hazard
).impact()

In [ ]:
# imp.plot_rp_imp((25,))
freq_curve = imp.calc_freq_curve() # impact exceedance frequency curve
freq_curve.plot();


In [ ]:
ent.impact_funcs.plot()

In [ ]:
haz = Hazard.from_excel(HAZARDS_DIR / "hazard_template.xlsx")

In [ ]:
haz.plot_intensity(0)

In [ ]:
dataset_infos = client.list_dataset_infos(
    properties={
        "country_name": ["Egypt", "Thailand"],
    },
)
dataset_infos

In [ ]:
haz = client.get_hazard("flood", properties={"country_name": "Thailand"}, dump_dir=HAZARDS_DIR)
haz

In [ ]:
handlers.generate_hazard_geojson(haz, 'Thailand')

In [ ]:
def sanitize_country_name(country_name):
    try:
        country = pycountry.countries.search_fuzzy(country_name)[0]
        return country.name
    except LookupError as exception:
        logger.log(
            "debug",
            f"Error while trying to sanitize country name. More info: {exception}",
        )
        raise ValueError(f"Failed to sanitize country name: {country_name}. More info: {exception}")
    
sanitize_country_name("thaeriland")

In [ ]:
def get_iso3_country_code(country_name: str):
    try:
        country_code = pycountry.countries.search_fuzzy(country_name)[0].alpha_3
        return country_code
    except Exception as exception:
        logger.log(
            "debug",
            f"Error while trying to match hazard time horizon datasets. More info: {exception}",
        )
        raise ValueError(
            f"Failed to retrieve ISO3 country code for country {country_name}. More info: {exception}"
        )


print(get_iso3_country_code("thailand"))
print(get_iso3_country_code("egypt"))